In [1]:
import ray.data
import pandas as pd
from ray.data.datasource import Partitioning
from kamiwaza.services.catalog.services import DataService
import os
import logging
os.environ['RAY_LOG_TO_DRIVER'] = '0'
ray.init(logging_level=logging.ERROR, ignore_reinit_error=True, include_dashboard=False)
logset = [logging.getLogger(x).setLevel(logging.ERROR) for x in [None, 'ray', 'ray.data']]



In [2]:
# Get dataset from Kamiwaza Catalog
catalog = DataService()
path = next(catalog.plugin.dataset_names(catalog.get_datasets('surface_data')))
print(path)

/Users/matt/code/weatherdata/data/surface_data


In [3]:
# Flag as partitioned by date for fast retrieval
partitioning = Partitioning(style='hive', field_names=['date'])
dataset = ray.data.read_parquet(path, partitioning=partitioning)
print(dataset)


Metadata Fetch Progress 0:   0%|          | 0.00/5.54k [00:00<?, ? task/s]

Parquet Files Sample 0:   0%|          | 0.00/10.0 [00:00<?, ? file/s]

Dataset(
   num_rows=1341573314,
   schema={
      date_time: timestamp[ns],
      station_id: string,
      station_number: int64,
      longitude: double,
      latitude: double,
      elevation: int64,
      state: string,
      country: string,
      priority: int64,
      pmsl: null,
      alti: null,
      tmpk: double,
      dwpk: null,
      sped: double,
      drct: double,
      cfrt: null,
      wwmo: null,
      date: string
   }
)


In [4]:
print(dataset.schema())

Column          Type
------          ----
date_time       timestamp[ns]
station_id      string
station_number  int64
longitude       double
latitude        double
elevation       int64
state           string
country         string
priority        int64
pmsl            null
alti            null
tmpk            double
dwpk            null
sped            double
drct            double
cfrt            null
wwmo            null
date            string


In [5]:
print(pd.DataFrame(dataset.take(5)))

Running 0: 0.00 row [00:00, ? row/s]

- ReadParquet 1: 0.00 row [00:00, ? row/s]

- limit=5 2: 0.00 row [00:00, ? row/s]

            date_time station_id  station_number  longitude  latitude  \
0 1933-01-01 15:00:00        RIV          722860    -117.25      33.9   
1 1933-01-01 16:00:00        RIV          722860    -117.25      33.9   
2 1933-01-01 17:00:00        RIV          722860    -117.25      33.9   
3 1933-01-01 18:00:00        RIV          722860    -117.25      33.9   
4 1933-01-01 19:00:00        RIV          722860    -117.25      33.9   

   elevation state country  priority  pmsl  alti        tmpk  dwpk  sped  \
0        466    CA      US         1  None  None  276.000000  None   1.5   
1        466    CA      US         1  None  None  279.299988  None   1.0   
2        466    CA      US         1  None  None  284.299988  None   1.0   
3        466    CA      US         1  None  None  286.500000  None   0.5   
4        466    CA      US         1  None  None  290.399994  None   1.0   

    drct  cfrt  wwmo        date  
0  180.0  None  None  1933-01-01  
1  160.0  None  None  1933-01-01  